In [8]:
!wget https://github.com/adoptium/temurin17-binaries/releases/download/jdk-17.0.11+9/OpenJDK17U-jdk_x64_linux_hotspot_17.0.11_9.tar.gz
!tar -xvzf OpenJDK17U-jdk_x64_linux_hotspot_17.0.11_9.tar.gz
!mv jdk-17.0.11+9 temurin17

--2025-11-29 18:53:09--  https://github.com/adoptium/temurin17-binaries/releases/download/jdk-17.0.11+9/OpenJDK17U-jdk_x64_linux_hotspot_17.0.11_9.tar.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/372925194/5d773fec-de9a-47d0-9b2f-344c8222da82?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-29T19%3A48%3A34Z&rscd=attachment%3B+filename%3DOpenJDK17U-jdk_x64_linux_hotspot_17.0.11_9.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-29T18%3A48%3A13Z&ske=2025-11-29T19%3A48%3A34Z&sks=b&skv=2018-11-09&sig=8fAETY13J8O%2BWpCKx8TL1LW6HKRbSMvEzshdO4wNQn0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4c

In [9]:
import os

os.environ["JAVA_HOME"] = "/content/temurin17"
os.environ["PATH"] += ":/content/temurin17/bin"

!java -version

openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [14]:
%%writefile SetPartitionBacktracking.java
public class SetPartitionBacktracking {

    private long calls = 0;

    public boolean canPartition(int[] nums) {
        int totalSum = 0;
        for (int num : nums) totalSum += num;
        if (totalSum % 2 != 0) return false;
        return backtrack(nums, 0, 0, totalSum / 2);
    }

    private boolean backtrack(int[] nums, int index, int currentSum, int target) {
        calls++;
        if (currentSum == target) return true;
        if (currentSum > target || index >= nums.length) return false;

        if (backtrack(nums, index + 1, currentSum + nums[index], target))
            return true;

        return backtrack(nums, index + 1, currentSum, target);
    }

    public long getCallCount() {
        return calls;
    }
}

Overwriting SetPartitionBacktracking.java


In [15]:
%%writefile ExperimentTimer.java
public class ExperimentTimer {

    public static long measureExecutionTime(Runnable algorithm) {
        long start = System.nanoTime();
        algorithm.run();
        long end = System.nanoTime();
        return end - start;
    }

    public static long runReplicas(Runnable algorithm, int replicas) {
        long total = 0;
        for (int i = 0; i < replicas; i++) {
            total += measureExecutionTime(algorithm);
        }
        return total / replicas;
    }
}

Overwriting ExperimentTimer.java


In [21]:
%%writefile Main.java
import java.util.Arrays;
import java.util.stream.IntStream;

public class Main {
    public static void main(String[] args) {

        int[] instance = IntStream.range(1, 7001).toArray();
        int replicas = 5;

        System.out.println("------------------------------------------------");
        System.out.println(" EXPERIMENTO SET PARTITION - BACKTRACKING");
        System.out.println(" IDE: GOOGLE COLAB");
        System.out.println("------------------------------------------------");

        long avgTime = ExperimentTimer.runReplicas(() -> {
            SetPartitionBacktracking spp = new SetPartitionBacktracking();
            spp.canPartition(Arrays.copyOf(instance, instance.length));
        }, replicas);

        System.out.println("Instancia: (tamaño = " + instance.length + ")");
        System.out.println("Réplicas: " + replicas);
        System.out.println("Tiempo promedio (s): " + avgTime / 1_000_000_000.0);
        System.out.println("Tiempo promedio (min): " + avgTime / 60_000_000_000.0);
    }
}

Overwriting Main.java


In [22]:
!javac Main.java ExperimentTimer.java SetPartitionBacktracking.java
!java Main

------------------------------------------------
 EXPERIMENTO SET PARTITION - BACKTRACKING
 IDE: GOOGLE COLAB
------------------------------------------------
Instancia: (tamaño = 7000)
Réplicas: 5
Tiempo promedio (s): 14.235174727
Tiempo promedio (min): 0.23725291211666666
